# Introduction #

We saw in the first lesson how to build an image classifier by attaching an untrained head of dense layers to a pretrained convolutional base. Since the pretrained base had already learned how to extract visual features, we could train an accurate image classifier with relatively little data.

In this lesson, we'll use the understanding we've gained of the layers in the convolutional base to improve this method even more. The key is in the hierarchy of visual features we learned about in the previous lesson.

# Using the Hierarchy of Features #

Let's review what we know about the feature extraction in the convolutional base.

The base is a sequence of layers that each perform a step in the extraction process. By the time the data has reached the classifier, it has undergone the extraction process many times. Simple features in the shallow layers combine and recombine to form the complex features in the deepest layers.

<!--TODO: feature extraction-->
<figure>
<img src="" width=400 alt="Feature extraction.">
</figure>


Because the network learns which features to extract to solve its classification problem, the features produced by the deepest layers will often closely resemble the training data.

The features produced by shallow layers, however, tend to be more generic and will generalize much better to a range of datasets.

For instance, suppose we train a convnet on images of dogs. The shallow layers will tend to produce features like lines and textures and contrasts, features useful on almost every problem. The deepest layers will tend to produce features resembling fur and eyes and snouts. It is unlikely these layers would be useful when classifying cars or flowers.

<!--TODO: optimal features-->
<figure>
<img src="" width=400 alt="Optimal features.">
</figure>


# Fine Tuning #

What this means is that when we are using a pretrained base as a feature extractor, it may be best to retrain some of its layers, starting with those closest to the output.

This technique is called **fine tuning**. It ensures that the layers most important for the classification have a chance to adapt to the new dataset.

In Lesson 1, we created a classifier by using a pretrained base with an untrained head of dense layers. During training, *only* the weights of the dense layers were updated. The the layers in the base were not trained; their weights we kept **frozen**.

To fine tune the model, we continue the training by **unfreezing** the layers in the base one-by-one until the model begins to **overfit**.

<!--TODO: overfitting-->
<figure>
<img src="" width=400 alt="Overfitting.">
</figure>

At that point, we know that unfreezing additional layers will be unlikely to improve the accuracy of the model on unseen data.

# Example - Training with Fine Tuning #

Let's work through an example to see how it goes. As before, let's assume we've already loaded a dataset with a training split `ds_train` and a validation split `ds_valid`.

In [ ]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.InteractiveSession(config=config)

import visiontools as vt
from visiontools import StanfordCars
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = '/home/jovyan/work/kaggle/datasets'

(ds_train, ds_valid), ds_info = tfds.load('stanford_cars/simple',
                                          split=['train', 'test'],
                                          shuffle_files=True,
                                          with_info=True,
                                          data_dir=DATA_DIR)

BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE
SIZE = [192, 192]
preprocess = vt.make_preprocessor(size=SIZE)

ds_train_ = (ds_train.map(preprocess)
            .cache()
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

ds_valid_ = (ds_valid.map(preprocess)
            .cache()
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

## Step 1 - Define Model ##

Now create a model just like you did in the first lesson, with a pretrained base and several dense layers as a head to act as the classifier.

In [ ]:
from tensorflow.keras import Sequential
import tensorflow.keras.layers as layers

from tensorflow.keras.applications import VGG16

pretrained_base = VGG16(include_top=False,
                        weights='imagenet',
                        input_shape=[192, 192, 3])
pretrained_base.trainable = False

model = Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

We're going to set up the training loop a little bit differently than before.

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=160,
    decay_rate=0.80,
    staircase=True,
)
optimizer = tf.keras.optimizers.Adam(lr_schedule)

# Number of epochs per round of training
EPOCHS = 5

## Step 2 - Train Classifier Head ##

Now, keeping all the layers in the base frozen, train the model until the validation loss no longer improves.

In [ ]:
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

history = model.fit(ds_train_,
                    validation_data=ds_valid_,
                    epochs=EPOCHS)

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

## Step 3 - Fine Tune the Base ##

Let's look at the layers in the base. We'll print out names and indices.

In [ ]:
for idx, layer in enumerate(pretrained_base.layers):
    print(idx, layer.name)

Let's tune the convolutional layers in block 5. This gives us indices, from the end backwards, `[17, 16, 15]`.

Iterate over top layers, tuning one by one. Since we've set the number of epochs per training round to `EPOCHS=5`, this will give us 5+3*5 = 20 epochs total.

In [ ]:
for r, idx in enumerate([17, 16, 15]):
    print("Unfreezing layer at index {}.".format(idx))
    pretrained_base.layers[idx].trainable = True

    # Recompile model after unfreezing a layer.
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )
    # Define epochs
    INIT, = history.epoch[-1] + 1, # start after last iteration's end
    TOTAL = history.epoch[-1] + 1 + EPOCHS
    # Retrain with layer at idx unfrozen.
    history = model.fit(
        ds_train_,
        validation_data=ds_valid_,
        initial_epoch=INIT,
        epochs=TOTAL,
    )
    # Concatenate this round's history to previous history
    history_frame = pd.concat([history_frame, pd.DataFrame(history.history)])

In [ ]:
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot();

# Conclusion #

<!--TODO: tut6 conclusion-->